In [ ]:
IMDB sentiment analysis

In [35]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data = pd.read_csv("IMDB Dataset.csv") 
# Preview the first 5 lines of the loaded data 
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [49]:
import re

mystr = data.iloc[1,0]
wordList = re.sub("[^\w]", " ",  mystr).split()
print(wordList)

['A', 'wonderful', 'little', 'production', 'br', 'br', 'The', 'filming', 'technique', 'is', 'very', 'unassuming', 'very', 'old', 'time', 'BBC', 'fashion', 'and', 'gives', 'a', 'comforting', 'and', 'sometimes', 'discomforting', 'sense', 'of', 'realism', 'to', 'the', 'entire', 'piece', 'br', 'br', 'The', 'actors', 'are', 'extremely', 'well', 'chosen', 'Michael', 'Sheen', 'not', 'only', 'has', 'got', 'all', 'the', 'polari', 'but', 'he', 'has', 'all', 'the', 'voices', 'down', 'pat', 'too', 'You', 'can', 'truly', 'see', 'the', 'seamless', 'editing', 'guided', 'by', 'the', 'references', 'to', 'Williams', 'diary', 'entries', 'not', 'only', 'is', 'it', 'well', 'worth', 'the', 'watching', 'but', 'it', 'is', 'a', 'terrificly', 'written', 'and', 'performed', 'piece', 'A', 'masterful', 'production', 'about', 'one', 'of', 'the', 'great', 'master', 's', 'of', 'comedy', 'and', 'his', 'life', 'br', 'br', 'The', 'realism', 'really', 'comes', 'home', 'with', 'the', 'little', 'things', 'the', 'fantasy', 

In [33]:
import re

mystr = str(data.iloc[[0,1]])
wordList = re.sub("[^\w]", " ",  mystr).split()
print(wordList)


['review', 'sentiment', '0', 'One', 'of', 'the', 'other', 'reviewers', 'has', 'mentioned', 'that', 'positive', '2', 'I', 'thought', 'this', 'was', 'a', 'wonderful', 'way', 'to', 'spend', 'ti', 'positive']


In [32]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
